pdf转txt

In [ ]:
import os

os.environ['LANGSMITH_TRACING_V2'] = 'true'

In [ ]:
import fitz


def extract_text_from_pdf(pdf_path):
    """提取pdf中的全部文本"""
    """Extract text from a PDF file."""
    text = ''
    with open(pdf_path, 'rb') as file:
        reader = pymupdf.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text()
    return text


pdf_path = 'data/rag/供电规则.pdf'  # Replace with your PDF file path
text = extract_text_from_pdf(pdf_path).replace('\n', '')
# Save the extracted text to a file
with open('data/extracted_text.txt', 'w', encoding='utf-8') as f:
    f.write(text)

In [ ]:
def recur_extract_images(pdf_path, out_path='pdf_images'):
    """递归提取文件夹中PDF中的图片"""
    import fitz  # PyMuPDF
    from pathlib import Path

    i = 0
    scr_path = Path(pdf_path)
    for pdf_path in scr_path.rglob('*.pdf'):
        doc = fitz.open(pdf_path)
        for page_index in range(len(doc)):
            page = doc[page_index]
            images = page.get_images(full=True)

            for img_index, img in enumerate(images):
                xref = img[0]  # 图像的对象引用号
                base_image = doc.extract_image(xref)
                image_bytes = base_image['image']
                image_ext = base_image['ext']
                with open(f'pdf_images/{i}.{image_ext}', 'wb') as f:
                    f.write(image_bytes)
            i += 1


recur_extract_images('e:/code/python/CnOCR-2.3.1/data/振兴')


In [ ]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

question = PromptTemplate.from_template('请根据以下内容回答问题：{context}问题：{question}，答案：{answer}')
examples = [
    {
        'context': '在中国，电力供应是由国家电网公司和南方电网公司负责的。',
        'question': '中国的电力供应由哪些公司负责？',
        'answer': '中国的电力供应由国家电网公司和南方电网公司负责。',
    },
    {
        'context': '供电规则包括用电安全、用电计量和用电价格等方面。',
        'question': '供电规则包括哪些方面？',
        'answer': '供电规则包括用电安全、用电计量和用电价格等方面。',
    },
]


prompt = FewShotPromptTemplate(examples=examples, example_prompt=question, suffix='请回答以下问题：{questio}')

In [ ]:
import os
from openai import OpenAI

client = OpenAI(
    api_key=os.getenv('DASHSCOPE_API_KEY'),
    base_url='https://dashscope.aliyuncs.com/compatible-mode/v1',
)


def ask(question: str):
    """Ask a question to the model."""
    completion = client.chat.completions.create(
        model='qwen-max',
        messages=[
            {'role': 'system', 'content': '你是'},
            {'role': 'user', 'content': question},
        ],
    )
    return completion.choices[0].message.content


In [ ]:
# 少量实例提示
examples = [
    {'fruit': 'apple', 'person': 'me', 'id': '1'},
    {'fruit': 'banana', 'person': 'you', 'id': '10'},
    {'fruit': 'cherry', 'person': 'him', 'id': '20'},
]

selecotr = SemanticSimilarityExampleSelector.from_examples(
    examples, OllamaEmbeddings(model='bge-m3:latest'), Chroma, k=1
)
# selector.select_examples({'id':24})
example_prompt = PromptTemplate.from_template('{person} like {fruit}.')
few_prompt = FewShotPromptTemplate(
    example_selector=selecotr, example_prompt=example_prompt, suffix='what fruit i like', input_variables=['id']
)
model = ChatOllama(model='qwen2.5:3b')